In [11]:
def dump_cell(cell, fname, nout, gid):
    from utils import io
    import struct
    
    with open(fname, 'wb') as f:
        # Header 
        # nout
        # galaxy id
        # number of cell
        f.write(struct.pack('i', 4))
        f.write(struct.pack('i', nout))
        f.write(struct.pack('i', 4))
        f.write(struct.pack('i', 4))
        f.write(struct.pack('i', gid))
        f.write(struct.pack('i', 4))
        f.write(struct.pack('i', 4))
        f.write(struct.pack('i', len(cell)))
        f.write(struct.pack('i', 4))
        for field in cell.dtype.names:
            io.write_fortran(f, cell[field])

            
def extract_gas(cell_all, gal, rscale=5.0):
    import numpy as np
    return _extract_cell(cell_all, gal['x'], gal['y'], gal['z'], rscale *gal['r'])


def _extract_cell(cell_all, xc, yc, zc, rr,
                  min_gas_density=0,
                  unit="code"):
    ind_c = np.where((np.square(cell_all['x'] - xc) + 
                      np.square(cell_all['y'] - yc) +
                      np.square(cell_all['z'] - zc)) < rr**2)[0]
    
    if min_gas_density > 0:
        ind_c = ind_c * (cell_all['var0'] > min_gas_density)

    cell = cell_all[ind_c].copy()
    # Match units to the GM galaxy output. 
    # position in Mpc
    # velocity in kms
    # mass in Msun (gal output originaly in 1e11Msun?)
    # Should I convert dx in physical unit too? 
    if unit == "physical":
        cell['x'] = (cell['x'] -0.5) * info.pboxsize
        cell['y'] = (cell['y'] -0.5) * info.pboxsize
        cell['z'] = (cell['z'] -0.5) * info.pboxsize
        cell['dx'] = cell['dx'] * info.pboxsize

    return cell

In [12]:
import load
import tree.halomodule as hmo
import numpy as np

base='./29172/'
nout = 180
s = load.sim.Sim(nout=nout, base=wdir, setup=True)
s.add_hydro(load=True)

info = load.info.Info(base=wdir, nout=nout)

gcat = hmo.Halo(base=wdir, is_gal=True, verbose=True, nout=nout)
hcat = hmo.Halo(base=wdir, is_gal=False, nout=nout)

data_dir = snapshots/
Ranges = [[0.0, 1.0], [0.0, 1.0], [0.0, 1.0]]

No AMR instance,
Loading one...
An AMR instance is created

Updating info.cpus
An AMR instance is created

Updating info.cpus
 Simulation set up.
An Hydro instance is created

 >>> working resolution (lmax) = 19
[Halo.load_info] loading info
[Halo.load_info] nout = 180, base =./29172/
[Halo.load_info] info is loaded
180
Loading file: ./29172/GalaxyMaker/gal/tree_bricks180
180


In [13]:
cluster = hcat.data[np.argmax(hcat.data['np'])]

In [14]:
def radial_cut(xc,yc,zc,r, x,y,z):
    return np.where(np.square(xc-x) +
                    np.square(yc-y) +
                    np.square(zc-z) < r**2)[0]

In [15]:
i_ok_gals = radial_cut(cluster['x'], cluster['y'], cluster['z'], 2.9* cluster['rvir'], gcat.data['x'], gcat.data['y'], gcat.data['z'])

out_dir = wdir + 'GalaxyMaker/CELL_00180/'

g_ok = gcat.data[i_ok_gals]

print(len(g_ok))

96


In [18]:
for gal in g_ok[:10]:
    #print(gal['x'], gal['y'], gal['z'], gal['r'])
    gcell = extract_gas(s.hydro.cell, gal)
    dump_cell(gcell, out_dir + "gal_cells_" + str(gal['id']).zfill(7), nout, gal['id'])
    print(max(gcell['var0']))
    print(min(gcell['x']), max(gcell['x']))

16826653.8381
0.498657226562 0.525024414062
16826653.8381
0.511306762695 0.513214111328
1989715.48446
0.509857177734 0.512542724609
4956167.32802
0.508590698242 0.515258789062
135.33817596
0.511138916016 0.512664794922
4956167.32802
0.510452270508 0.513519287109
287994.856149
0.510406494141 0.512313842773
885212.046825
0.508361816406 0.511505126953
214.275493232
0.515930175781 0.517471313477
4956167.32802
0.509857177734 0.51188659668


In [17]:
gcell.dtype

dtype([('x', '<f8'), ('y', '<f8'), ('z', '<f8'), ('dx', '<f8'), ('var0', '<f8'), ('var1', '<f8'), ('var2', '<f8'), ('var3', '<f8'), ('var4', '<f8'), ('var5', '<f8')])

In [118]:
import matplotlib.pyplot as plt

In [18]:
gal['id']

4

In [125]:
gc = gcell[range(0, len(gcell), 100)]

plt.scatter(gc['x'], gc['y'])
plt.show()

In [20]:
from utils import io

<module 'utils.io' from '/home/hoseung/Work/pyclusterevol/utils/io.py'>

In [33]:
f = open(out_dir + "gal_cells_0000010", 'rb')
nout = io.read_fortran(f, dtype=np.int32, check=False)[0]
gid = io.read_fortran(f, dtype=np.int32, check=False)[0] 
ncell = io.read_fortran(f, dtype=np.int32, check=False)[0]
x  = io.read_fortran(f, dtype=np.float64, n=ncell) # guess array length.
y  = io.read_fortran(f, dtype=np.float64, n=ncell)
z  = io.read_fortran(f, dtype=np.float64, n=ncell)
dx = io.read_fortran(f, dtype=np.float64, n=ncell)
v0 = io.read_fortran(f, dtype=np.float64, n=ncell)
v1 = io.read_fortran(f, dtype=np.float64, n=ncell)
v2 = io.read_fortran(f, dtype=np.float64, n=ncell)
v3 = io.read_fortran(f, dtype=np.float64, n=ncell)
v4 = io.read_fortran(f, dtype=np.float64, n=ncell)
v5 = io.read_fortran(f, dtype=np.float64, n=ncell)

In [35]:
gid

10

In [89]:
f.close()